In [40]:
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers,CRF
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite import metrics
from sklearn.model_selection import cross_val_predict, cross_val_score

import pandas as pd
import numpy as np
import csv

In [41]:
def ez_pandas_reader(fname):
    df = pd.read_csv(fname)
    df = df.set_index("id")
    return df

In [42]:
def convert(path):
    vals = []
    with open(path, 'r',) as f:
        reader = csv.reader(f, delimiter='\t')
        temp = []
        next(reader)
        for line in enumerate(reader):
            if not line[1]:
                vals.append(temp)
                temp = []
            else:
                t = [ '{}'.format(x) for x in list(csv.reader([line[1][0]], delimiter=',', quotechar='"'))[0]]
                x = [t[1], "", t[2]]
                temp.append(tuple(x))
        vals.append(temp)
    return vals

In [43]:
def test_convert(path):
    vals = []
    with open(path, 'r',) as f:
        reader = csv.reader(f, delimiter='\t')
        temp = []
        next(reader)
        for line in enumerate(reader):
            if not line[1]:
                vals.append(temp)
                temp = []
            else:
                t = [ '{}'.format(x) for x in list(csv.reader([line[1][0]], delimiter=',', quotechar='"'))[0] ]
                temp.append(tuple([t[1],""]))
        vals.append(temp)
    return vals

In [44]:
train = convert("/Users/johnsonchan/Documents/Coding/Class/CMPSC 190I W/HW 4/HW4_spanish_ner_data/train.csv")

In [45]:
validate = convert("/Users/johnsonchan/Documents/Coding/Class/CMPSC 190I W/HW 4/HW4_spanish_ner_data/validation.csv")

In [46]:
test = test_convert("/Users/johnsonchan/Documents/Coding/Class/CMPSC 190I W/HW 4/HW4_spanish_ner_data/test_noans.csv")

In [47]:
labelToType = {
    "B-LOC": 0,
    "B-MISC": 1,
    "B-ORG": 2,
    "B-PER": 3,
    "I-LOC": 4,
    "I-MISC": 5,
    "I-ORG": 6,
    "I-PER": 7,
    "O": 8
}

In [48]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word.isspace=%s' % word.isspace(),
        'word.isupper=%s' % word.isupper(),
        'word.islower=%s' % word.islower(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    features.extend([
        'word.lower=' + word.lower(),
        'word.upper=' + word.upper(),
        'word.title=' + word.title(),
        'word[:2]=' + word[:2],
        'word[:3]=' + word[:3],
        'word[:4]=' + word[:4],
        'word[-4:]=' + word[-4:],
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
    ])
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    if i > 1:
        word2 = sent[i-2][0]
        postag2 = sent[i-2][1]
        
        # Features for the previous two words
        features.extend([
            '-2:word.lower=' + word2.lower(),
            '-2:word.istitle=%s' % word2.istitle(),
            '-2:word.isupper=%s' % word2.isupper(),
            '-2:postag=' + postag2,
            '-2:postag[:2]=' + postag2[:2],
        ])
    if i < len(sent) - 2:
        word3 = sent[i+2][0]
        postag3 = sent[i+2][1]
        
        # Features for the word two positions ahead
        features.extend([
            '+2:word.lower=' + word3.lower(),
            '+2:word.istitle=%s' % word3.istitle(),
            '+2:word.isupper=%s' % word3.isupper(),
            '+2:postag=' + postag3,
            '+2:postag[:2]=' + postag3[:2],
        ])
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [49]:
X_train = [sent2features(s) for s in train]
y_train = [sent2labels(s) for s in train]

X_test = [sent2features(s) for s in validate]
y_test = [sent2labels(s) for s in validate]

X_ans = [sent2features(s) for s in test]

In [11]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.1),
    'c2': scipy.stats.expon(scale=0.01),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels, zero_division=1)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=10,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=100,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

In [20]:
labels = ["B-LOC", "B-MISC","B-ORG", "B-PER", "I-LOC", "I-MISC","I-ORG","I-PER"]

In [21]:
crf_new = rs.best_estimator_
y_pred_n = crf_new.predict(X_test)
metrics.flat_f1_score(y_test, y_pred_n,
                      average='weighted', labels=labels)

0.7698380092466075

In [22]:
y_pred_final_n = crf_new.predict(X_ans)

In [23]:
test_noans = ez_pandas_reader("/Users/johnsonchan/Documents/Coding/Class/CMPSC 190I W/HW 4/HW4_spanish_ner_data/test_noans.csv")

In [38]:
y_pred_to_label_n = []
for i in y_pred_final_n:
    for j in i:
        y_pred_to_label.append(labelToType[j])

In [39]:
test_ans = test_noans.copy()
for i in range(0, len(y_pred_to_label)):
    test_ans.loc[i, "label"] = str(y_pred_to_label[i])
test_ans["label"].to_csv("/Users/johnsonchan/Documents/Coding/Class/CMPSC 190I W/HW 4/HW4_spanish_ner_data/test_answer.csv", header=True)